In [1]:
from FC_RNN_Evaluater.FC_RNN_Evaluater import *
from FC_RNN_Evaluater.Stateful_FC_RNN_Configuration import *
from FC_RNN_Evaluater.runFC_RNN_Experiment import *

/home/mcicek/anaconda3/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters
Using TensorFlow backend.


In [2]:
from keras import Model
from keras.layers import TimeDistributed, LSTM, Dense, Dropout, Flatten, Input

In [7]:
def getFinalModel(timesteps = timesteps, lstm_nodes = lstm_nodes, lstm_dropout = lstm_dropout, 
                  lstm_recurrent_dropout = lstm_recurrent_dropout, num_outputs = num_outputs, 
                  lr = learning_rate, include_vgg_top = include_vgg_top, use_vgg16 = use_vgg16):

    inp, cnn_model, modelID, preprocess_input = getCNN_Model(use_vgg16 = use_vgg16)

    #finalModel = getLSTM_Model(inp, cnn_model, lstm_nodes = lstm_nodes, lstm_dropout = lstm_dropout, lstm_recurrent_dropout = lstm_recurrent_dropout)
    #finalModel.add(Dense(num_outputs))

    auxiliary_input = TimeDistributed(Input(shape=(num_outputs, ), name='aux_input')) # train_batch_size, timesteps
    x = (concatenate([TimeDistributed(cnn_model).output, auxiliary_input]))#
    midModel = Model(inputs=[cnn_model.input, auxiliary_input], outputs=x)
    lstm_out = LSTM(lstm_nodes, dropout=lstm_dropout, recurrent_dropout=lstm_recurrent_dropout, stateful=True)(x.output)
    main_output = Dense(num_outputs)(lstm_out)
    #finalModel = getLSTM_Model(inp, midModel, lstm_nodes = lstm_nodes, lstm_dropout = lstm_dropout, lstm_recurrent_dropout = lstm_recurrent_dropout)
    finalModel = Model(inputs=midModelmodel.input, outputs=main_output)

    #finalModel = Sequential()
    #finalModel.add(TimeDistributed(midModel,  input_shape=(train_batch_size, timesteps, 1027), name = 'tdCNN')) 
    #finalModel.add(LSTM(lstm_nodes, dropout=lstm_dropout, recurrent_dropout=lstm_recurrent_dropout, stateful=True))#, activation='relu'
    #finalModel.add(Dense(num_outputs))

    adam = Adam(lr=lr)
    finalModel.compile(optimizer=adam, loss='mean_absolute_error') #'mean_squared_error', metrics=['mae'])# 

    modelID = modelID + '_seqLen%d' % timesteps; modelID = modelID + '_stateful'; modelID = modelID + '_lstm%d' % lstm_nodes
    modelID = modelID + '_output%d' % num_outputs; modelID = modelID + '_BatchSize%d' % train_batch_size
    modelID = modelID + '_inEpochs%d' % in_epochs; modelID = modelID + '_outEpochs%d' % out_epochs
    modelID = modelID + '_AdamOpt_lr-%f' % lr; modelID = modelID + '_%s' % now()[:-7].replace(' ', '_').replace(':', '-')
    
    return cnn_model, finalModel, modelID, preprocess_input


In [8]:
vgg_model, full_model, modelID, preprocess_input = getFinalModel(timesteps = timesteps, lstm_nodes = lstm_nodes, lstm_dropout = lstm_dropout, lstm_recurrent_dropout = lstm_recurrent_dropout, 
                      num_outputs = num_outputs, lr = learning_rate, include_vgg_top = include_vgg_top)

AttributeError: Layer time_distributed_6 has no inbound nodes.

In [3]:
full_model = trainCNN_LSTM(full_model, modelID, out_epochs, trainingSubjects, timesteps, output_begin, num_outputs, 
                  batch_size = train_batch_size, in_epochs = in_epochs, stateful = STATEFUL, preprocess_input = preprocess_input)

All frames and annotations from 1 datasets have been read by 2019-01-29 00:45:23.596839
1. set (Dataset 9) being trained for epoch 1 by 2019-01-29 00:45:32.509373!
Epoch 1/1
882/882 [==============================] - 27s 31ms/step - loss: 0.3977
Epoch 1 completed!


In [12]:
def unscaleEstimations(test_labels, predictions, scalers, output_begin, num_outputs):
    """* label_rescaling_factor * label_rescaling_factor
    """
    sclrs = [scalers[0][output_begin:output_begin+num_outputs], scalers[1][output_begin:output_begin+num_outputs]]
    test_labels = unscaleAnnoByScalers(test_labels, sclrs)
    predictions = unscaleAnnoByScalers(predictions, sclrs)
    return test_labels, predictions

In [13]:
def evaluateSubject(full_model, subject, test_gen, test_labels, timesteps, output_begin, num_outputs, angles, batch_size, stateful = False, record = False):
    if num_outputs == 1: angles = ['Yaw']
    printLog('For the Subject %d (%s):' % (subject, BIWI_Subject_IDs[subject]), record = record)
    predictions = full_model.predict_generator(test_gen, steps = int(len(test_labels)/batch_size), verbose = 1)
    #kerasEval = full_model.evaluate_generator(test_gen) 
    test_labels, predictions = unscaleEstimations(test_labels, predictions, BIWI_Lebel_Scalers, output_begin, num_outputs)
    full_model.reset_states()
    outputs = []
    for i in range(num_outputs):
        if stateful:
            start_index = (test_labels.shape[0] % batch_size) if batch_size > 1 else 0
            matrix = numpy.concatenate((test_labels[start_index:, i:i+1], predictions[:, i:i+1]), axis=1)
            differences = (test_labels[start_index:, i:i+1] - predictions[:, i:i+1])
        else:
            print(test_labels[:, i:i+1].shape, predictions[:, i:i+1].shape)
            matrix = numpy.concatenate((test_labels[:, i:i+1], predictions[:, i:i+1]), axis=1)
            differences = (test_labels[:, i:i+1] - predictions[:, i:i+1])
        absolute_mean_error = np.abs(differences).mean()
        printLog("\tThe absolute mean error on %s angle estimation: %.2f Degree" % (angles[i], absolute_mean_error), record = record)
        outputs.append((matrix, absolute_mean_error))
    return full_model, outputs


In [14]:
def evaluateCNN_LSTM(full_model, label_rescaling_factor, testSubjects, timesteps, output_begin, 
                     num_outputs, batch_size, angles, stateful = False, record = False, preprocess_input = None):
    if num_outputs == 1: angles = ['Yaw']
    test_generators, test_labelSets = getTestBiwiForImageModel(testSubjects, timesteps, False, output_begin, num_outputs, 
                                            batch_size = batch_size, stateful = stateful, record = record, preprocess_input = preprocess_input)
    results = []
    for subject, test_gen, test_labels in zip(testSubjects, test_generators, test_labelSets):
        full_model, outputs = evaluateSubject(full_model, subject, test_gen, test_labels, timesteps, output_begin, num_outputs, angles, batch_size = batch_size, stateful = stateful, record = record)
        results.append((subject, outputs))
    means = evaluateAverage(results, angles, num_outputs, record = record)
    return full_model, means, results 


In [15]:
 full_model, means, results = evaluateCNN_LSTM(full_model, label_rescaling_factor = label_rescaling_factor, 
                     testSubjects = testSubjects, timesteps = timesteps,  output_begin = output_begin, 
                    num_outputs = num_outputs, batch_size = test_batch_size, angles = angles, stateful = STATEFUL, preprocess_input = preprocess_input)

All frames and annotations from 1 datasets have been read by 2019-01-29 00:52:13.451180
For the Subject 9 (M03):
882/882 [==============================] - 12s 14ms/step
	The absolute mean error on Pitch angle estimation: 20.35 Degree
	The absolute mean error on Yaw angle estimation: 29.29 Degree
	The absolute mean error on Roll angle estimation: 31.73 Degree


In [10]:
test_generators, test_labelSets = getTestBiwiForImageModel(testSubjects, timesteps, False, output_begin, num_outputs, 
                                            batch_size = 1, stateful = True, preprocess_input = preprocess_input)

All frames and annotations from 1 datasets have been read by 2019-01-28 23:01:54.311180


In [11]:
test_gen, test_labels = test_generators[0], test_labelSets[0]

In [12]:
test_labels

array([[1.81540612, 1.01903605, 1.20626227],
       [1.81104456, 1.02161907, 1.21798137],
       [1.8071115 , 1.02693495, 1.2186752 ],
       ...,
       [1.86528669, 1.18353251, 1.28262154],
       [1.86317814, 1.19736085, 1.28078107],
       [1.85303369, 1.19928671, 1.2678096 ]])